In [265]:
import pandas as pd
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [266]:
biden_train = pd.read_csv('biden_stance_train_public.csv')
biden_test = pd.read_csv('biden_stance_test_public.csv')
#trump = pd.read_csv('trump_stance_train_public.csv')

biden.head()
len(biden)
biden['label'].value_counts()

## Data Cleaning

#### Stage 1: case to lower

In [5]:
biden['text'] = biden['text'].str.lower()

0      imma keep it real with y’all i don’t think the...
1      @user guess the #cult45 #trumpdrunkmorons can ...
2      joe “let’s be reasonable guys not everybody ca...
3      pelosi says biden shouldn't debate trump: 'i w...
4      @user hey @user, here’s what your own party th...
                             ...                        
870    i’m excited to hear from hunter biden’s family...
871    @user your still talking about trump? get over...
872    @user we will...thank you for checking\n#biden...
873    i thought you were not doing bad, but then the...
874    @user exactly @user !!! we all must vote in pe...
Name: text, Length: 875, dtype: object

#### Stage 2: tokenization

In [69]:
from nltk.tokenize import RegexpTokenizer

In [70]:
tokenizer = RegexpTokenizer(r'\w+')
#tokens = tokenizer.tokenize(biden['text'][0])

In [71]:
biden['word_tok'] = biden['text'].apply(lambda x: tokenizer.tokenize(x))

tokenizer = RegexpTokenizer(r'\w+'')
tokens = tokenizer.tokenize(text)
                            
def word_token(x): #x=biden['text_tok'][0]
    word_tok = []
    for word in x:
        word = word_tokenize(word)
        word_tok.append(word)
    return word_tok

from nltk.tokenize import sent_tokenize, word_tokenize

biden['text'][0]

sent_tokenize(biden['text'][0])

biden['text_tok'] = biden['text'].apply(lambda x: sent_tokenize(x))

def word_token(x): #x=biden['text_tok'][0]
    word_tok = []
    for word in x:
        word = word_tokenize(word)
        word_tok.append(word)
    return word_tok

biden['word_tok'] = biden['text_tok'].apply(lambda x: word_token(x))

#### Stage 3: regular expression (remove special characters)

import re

def regu_exp(x):
    clean_text = []
    for words in x:
        clean = []
        for w in words:
            res = re.sub(r"[^\w\s]" ,"",w)
            if res != '':
                clean.append(res)
        clean_text.append(clean)
    return clean_text

biden['clean_text1'] = biden['word_tok'].apply(lambda x: regu_exp(x))

#### Stage 4: Stop word removal

In [81]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
from nltk.corpus import stopwords

In [85]:
def stopword(x): 
    clean_text = []
    for word in x:
        if not word in stopwords.words('english'):       
            clean_text.append(word)
    return clean_text

In [86]:
biden['clean_text2'] = biden['word_tok'].apply(lambda x: stopword(x))

#### Stage 5: Stemming

In [42]:
from nltk.stem.porter import PorterStemmer

In [43]:
port = PorterStemmer()

In [88]:
def stem(x):
    clean_text = []
    for word in x:
        word = port.stem(word)
        clean_text.append(word)
    return clean_text

In [89]:
biden['clean_text3'] = biden['clean_text2'].apply(lambda x: stem(x))

#### lemitization

In [91]:
from nltk.stem.wordnet import WordNetLemmatizer

In [92]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [93]:
def lemmatize(x):
    lem = []
    for word in x:
        lem.append(wnet.lemmatize(word))
    return lem

In [94]:
biden['clean_text4'] = biden['clean_text3'].apply(lambda x: stem(x))

In [267]:
biden_train

,tweet_id,text,label
0,1311162417938919424,imma keep it real with y’all i don’t think the...,NONE
1,1299361296400429057,@USER guess the #cult45 #trumpdrunkmorons can ...,NONE
2,1239350672782483456,joe “let’s be reasonable guys not everybody ca...,NONE
3,1299022276571140099,pelosi says biden shouldn't debate trump: 'i w...,AGAINST
4,1295359993655578624,"@USER hey @USER, here’s what your own party th...",NONE
...,...,...,...
870,1296637794966745088,i’m excited to hear from hunter biden’s family...,NONE
871,1295317559802302466,@USER your still talking about trump? get over...,AGAINST
872,1292661159078723590,@USER we will...thank you for checking\n#biden...,FAVOR
873,1298457178773184512,"i thought you were not doing bad, but then the...",NONE


#### words to string

In [133]:
# Python program to convert a list
# to string using list comprehension
def listtoStr(s):
    Str = ' '.join([str(elem) for elem in s])
    return Str

In [134]:
biden['clean_text5'] = biden['clean_text4'].apply(lambda x: listtoStr(x))

In [135]:
x_clean = biden['clean_text5'].tolist()

#### Function to integrate all stage

In [193]:
from nltk.tokenize import RegexpTokenizer

In [194]:
from nltk.stem.porter import PorterStemmer

In [195]:
from nltk.stem.wordnet import WordNetLemmatizer

In [196]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [197]:
from nltk.corpus import stopwords

In [198]:
tokenizer=RegexpTokenizer(r'\w+')
en_stopwords= set(stopwords.words('english'))
ps = PorterStemmer()

In [199]:
def getCleanedText(text):
    text = text.lower()
    
    #tokenize
    tokens = tokenizer.tokenize(text)
    new_tokens = [token for token in tokens if token not in en_stopwords]
    
    stemmed_tokens = [ps.stem(tokens) for tokens in new_tokens]
    
    clean_text = " ".join(stemmed_tokens)
    
    return clean_text

In [200]:
x_clean = [getCleanedText(i) for i in biden_train['text']]
xt_clean = [getCleanedText(i) for i in biden_test['text']]

In [237]:
#x_clean

['imma keep real think debates2020 twitch',
 'user guess cult45 trumpdrunkmoron pick poison unlik jim jone offer one trump serv assort trumpgenocid ettd votebluetosaveamerica',
 'joe let reason guy everybodi get medicar biden demdeb',
 'pelosi say biden debat trump legitim convers http knew would happen noth make rise basement left scratch bite eat rather face presid maga',
 'user hey user parti think billboard paid user world see user win 2020elect http',
 'user biden job control trump moder job trump follow rule agre upon campaign chri wallac control trump biden took high road',
 'user nanci pelosi other democrat leader think american peopl stupid rude un respect againt peopl lier user trump trump2020 maga kag2020 fourmoreyear',
 'get sander v warren battl yet seen everi fight warren need take glove sander comment onlin support demdeb',
 'remind local race matter ever take minut today confirm updat voter registr get info upcom elect make sure plan vote 2020elect massachusett votema h

## Vectorization

#### x

In [252]:
from sklearn.feature_extraction.text import CountVectorizer

In [253]:
vect =  CountVectorizer(ngram_range=(1,2))

In [254]:
x_vec = vect.fit_transform(x_clean).toarray()

In [168]:
print(vect.get_feature_names())

['00', '00 2020', '00 et', '000', '000 americsm', '000 dead', '000 dow', '000 soul', '000 top', '0626', '0626 increas', '10', '10 20', '10 blade', '10 hr', '10 point', '10 say', '100', '100 emot', '100 gone', '100 million', '1000', '1000 boat', '100daysdemocratswaterloo', '100daysdemocratswaterloo get', '10x1000', '10x1000 live', '11', '11 must', '11thhour', '11thhour bidenharri', '125', '125 000', '13', '13 00', '13 biden', '13 million', '13887', '13887 prepar', '14', '14 mile', '15', '15 flew', '150', '150 000', '1500', '1500 peopl', '16', '16 http', '16 never', '16 year', '160', '160 000', '17', '17 time', '18', '18 2020', '18 american', '1816', '1816 votetrumpout', '18th', '18th 2020', '19', '19 biden', '19 deathocrat', '19 economi', '19 factor', '1984', '1984 us', '1a', '1a liber', '1st', '1st bill', '1st done', '20', '20 say', '20 screw', '20 youtub', '200', '200 000', '200 million', '200k', '200k american', '200k dead', '200k fail', '2010', '2016', '2016 2020', '2016 demconventi

In [255]:
xt_vec = vect.transform(xt_clean).toarray()

#### y

In [243]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

In [268]:
lbl_enc = preprocessing.LabelEncoder()
y_train = lbl_enc.fit_transform(biden_train['label'].values)
y_test = lbl_enc.transform(biden_test['label'].values)

## Multinomial Naive Bayes

p(y=1|x) probability that the email belongs to spam
p(y=0|x) probability that the email belongs to non-spam
x=[text]=[x1,x2,x3,...,xn]

Bayes theorem:  
p(y=1|x) = p(x|y=1)p(y=1)/p(y=0)
p(y=0|x) = p(x|y=0)p(y=0)/p(y=1)  
case x give y=0 is hard to get

Naive Bayes:  
given a particular class value(1/0), the value of a particular feature is independent of the value of other features

p(x|y=1) = p(x1|y=0)p(x2|y=0)...p(xn|y=0)


In [270]:
from sklearn.naive_bayes import MultinomialNB

In [271]:
classifier = MultinomialNB()

In [272]:
classifier.fit(x_vec,y_train)

MultinomialNB()

In [257]:
y_pred = classifier.predict_proba(xt_vec)


array([[3.47386759e-01, 3.02599722e-02, 6.22353269e-01],
       [1.31754064e-03, 9.64952717e-01, 3.37297421e-02],
       [1.90813907e-06, 3.16019502e-06, 9.99994932e-01],
       ...,
       [6.10168834e-04, 9.77503860e-01, 2.18859713e-02],
       [8.93540173e-05, 9.99839760e-01, 7.08859107e-05],
       [8.45178012e-01, 1.52245055e-01, 2.57693279e-03]])

In [273]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(xt_vec)
acc = accuracy_score(y_test, y_pred)

In [274]:
acc

0.6613333333333333

In [275]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[76,  9, 21],
       [16, 78, 18],
       [35, 28, 94]], dtype=int64)